In [122]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from basic.claims import connect #, get_claims, get_users, get_kibana_gate, products_bankiros, nulls, get_medium, get_claims_raw
# from basic.external import cpa_parse, get_yandex_data
# from basic.dscore import cre_get, mfo_score
from basic.reports import cre_keyword_report, master_report, registrations_medium
from basic.vars import regions_mapper, claim_offers
from bson.objectid import ObjectId
import requests
import json
from datetime import datetime, date, timedelta
import pandas as pd
from time import sleep
from base64 import b64encode
import numpy as np
# from robobrowser import RoboBrowser
import re
# import csv, sqlite3
# import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
from calendar import monthrange
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1000)
# from IPython.display import Filelink, FileLinks


client, es = connect()

Доспуп к Эластику 	 True
Доспуп к Монге 		 True


In [123]:
#pip install numpy --upgrade.

In [124]:
# !pip install pickledb pymongo elasticsearch==6.8 oauth2client gspread df2gspread \
# robobrowser beautifulsoup4 lxml yourls Werkzeug==0.16.1 paramiko xlsxwriter pyminizip \
# html5lib zodiac_sign python-pptx plotly df2gspread gspread

In [125]:
# !pip install gspread_dataframe

def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)    
            
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)  # сюда json файл если он не в той же папке
            creds = flow.run_local_server(port=0)        
            
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)    
    return creds

def pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=RANGE_NAME).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data
    
def to_float(x):
    try:
        return(float(x.replace("\xa0", '').replace(" ", '').strip()))
    except Exception as e:
        return np.nan

def cre_get(es, since):
    
    chunk = 10000
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    payload = {"size": chunk,
               "_source": ["params.parsed"],
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword": "CRE"}},
                         {"range":{"timestamp":{"gte":gte,"format":"epoch_millis"}}}
                        ]}},
"_source": ["params.parsed", "timestamp"]
              }
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=3000)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/chunk)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=3000)
        hits = hits + d['hits']['hits']
        print(doc_count-chunk*i, end ="; ", flush=True)
        sleep(0.5)
#         break
    print(len(hits), 'raz')
    cre = []
    for h in hits:
        if len(h['_source']['params']['parsed'])>10:
            h2 = json.loads(h['_source']['params']['parsed'].replace("'", '"'))
            h2.update({"timestamp": pd.to_datetime(h['_source']['timestamp'])})
            cre.append(h2)
    df = pd.DataFrame(cre).sort_values("timestamp", 
                                       ascending=False).drop_duplicates('userId', keep='first').reset_index(drop=True)
    df.index = df['userId']
      
            
    return df['score'].to_dict()

def mm_requestId(x):
    if type(x)==dict and 'NOTE' in x.keys():
        if x['NOTE'].find('HTTP_RESPONSE_BODY') >0:
            htr = json.loads(x['NOTE'][x['NOTE'].find('HTTP_RESPONSE_BODY')+20:])
            htr = htr['requestId'] if 'requestId' in htr.keys() else np.nan
#             print(htr)
        else: 
            htr = np.nan
    else: 
        htr = np.nan
    return htr

def get_exact_claims(since, till, client):
    req = {"createdAt": {"$gte": datetime.combine(since, datetime.min.time()) - timedelta(hours=3),
                          "$lt": datetime.combine(till, datetime.min.time())- timedelta(hours=3)},
#            "product._id": {"$in": [ObjectId("5e9846be79dd27000141f476"),
# ObjectId("5ef4473acf22a90001010445")]},
           #"product._id": {"$in": [ObjectId(product) for product in products]}}
            "status": {"$nin": [ "ERROR", "NEW1"]}} 
    results_count = client['claim-service']['claims'].count_documents(req)
    # print(req)
    print(results_count)
    all_ans = []
    batch_size = 10000
    for i in range(results_count//batch_size+1):
        ans = list(client['claim-service']['claims'].find(req, {"product": 1, "partner":1,
                                                                "borrower": 1, "statusFields":1,
                                                                "status": 1, "adapterName": 1,
                                                                "createdAt": 1, 'type':1,
                                                                "updatedAt":1, "clickId": 1,
                                                                "whence":1}).limit(batch_size).skip(i*batch_size))
        all_ans = all_ans + ans
    #ans = trans(all_ans, client)
    df = pd.DataFrame(all_ans)
    df['id'] = df["_id"].apply(lambda x: str(x))
    df['dt'] = df["createdAt"].apply(lambda x: (x+timedelta(hours=3)).date())
    df['productId'] = df['product'].apply(lambda x: str(x['_id']))
    df['product_type'] = df['product'].apply(lambda x: x['productType'] if 'productType' in x.keys() else np.nan)
    df['product_name'] = df['product'].apply(lambda x: x['name'])
    df['partner'] = df['partner'].apply(lambda x: x['name'])
    df['userId'] = df['borrower'].apply(lambda x: x['userId'])
    df['phone'] = df['borrower'].apply(lambda x: x['phone'] if 'phone' in x.keys() else np.nan)

    return df


def get_kibana_claims(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"CLAIM"}},
#                          {"match_phrase":{"params.sourceGlobal.keyword":{"query": sourceGlobal }}},
                         {"range":{"timestamp":{"gte":gte,"format":"epoch_millis"}}}
                        ]}},
"_source": ["params.parsed",'params.clickId', 'params.userId', 'params.utm_campaign', 'params.utm_medium', 
            'params.utm_content','params.utm_source', 'params.utm_term', 'params.gaId', 'params.yandexClientID',
            'params.sourceGlobal',"timestamp"]}
              
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
                  'transaction_id', 'clickId', 'sourceGlobal', 'utm_content','orderId','transactionId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

def get_kibana_postback(es, since):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(date.today()+timedelta(days=1), datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"POSTBACK"}},
#                          {"match_phrase":{"params.whence.keyword":{"query": whence }}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', 'payout', 'status',
                'transaction_id', 'externalId','orderId','transactionId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

In [126]:
since = date(2021,12,1)
till = date(2022,1,1)

In [127]:
df = get_exact_claims(since, till, client)
df['requestId'] = df[df['partner']=='MoneyMan']['statusFields'].apply(mm_requestId)
dpbk = get_kibana_postback(es, since)
def float_apply(x):
    try:
        return float(x)
    except ValueError:
        return None
dpbk['payout'] = dpbk['payout'].apply(float_apply)
dpbk['payout'] = dpbk.apply(lambda x: x['payout']*1.2 if x['partnerName']=='admitad' else x['payout'], axis=1)
dpbk = dpbk[dpbk['status'].isin(['APPROVED', 'approved', 'ISSUED'])].sort_values('payout').drop_duplicates('transitionId', keep='last')
dr = pd.merge(left=df, right=dpbk, left_on='id', right_on='transitionId', how='left')
dc = get_kibana_claims(es, since,till)[['clickId', 'userId', 'utm_campaign', 'utm_medium', 'utm_content', 
                                        'utm_source', 'utm_term', 'gaId', 'yandexClientID', 'sourceGlobal']]
dr = pd.merge(left=dr, right=dc[dc['clickId'].notnull()].drop_duplicates('clickId'), on='clickId', how='left')

cre = cre_get(es, since)
dr['score'] = dr['userId_x'].map(cre).fillna(0).astype(int)

dr.to_pickle(f'reports/{datetime.strftime(since, "%B")}_claimReport.pickle.gz')
dc.to_pickle(f'reports/{datetime.strftime(since, "%B")}_claims.pickle.gz')

def AR_count(score, productType, status):
    koef = 1
    check = [2200, 1700, 1100, 5500]
    ARlist = [[0.00499500499500499, 0.00305343511450382, 0.0714285714285714, 0.0], 
              [0.00380818053596615, 0.00871508379888268, 0.0768345323741007, 0.0], 
              [0.00685378590078329, 0.0136418349605457, 0.0846340617412418, 0.0053475935828877], 
              [0.0111028867505552, 0.0255610972568579, 0.0980751604032997, 0.0116279069767442], 
              [0.00938967136150235, 0.0349985043374215, 0.126042632066728, 0.0], 
              [0.0109689213893967, 0.0407338308457712, 0.140204271123491, 0.0], 
              [0.0141884222474461, 0.0514050719671008, 0.113486842105263, 0.0188679245283019], 
              [0.0150753768844221, 0.0687361419068736, 0.0743243243243243, 0.0], 
              [0.00943396226415094, 0.0415430267062315, 0.0892857142857143, 0.0],
              [0.00943396226415094, 0.0415430267062315, 0.0892857142857143, 0.0]]
    
    if score is np.nan or status in ['NEW', 'ERROR']:
        return 0
    elif score > 10 and productType == 'CASH_LOAN':
        return round(ARlist[int(round(score, -1)/10)-1][0]*check[0]*koef, 2)
    elif score > 10 and productType == 'CREDIT_CARD':
        return round(ARlist[int(round(score, -1)/10)-1][1]*check[1]*koef, 2)
    elif score > 10 and productType == 'LOAN':
#         print(score, productType, status)
        return round(ARlist[int(round(score, -1)/10)-1][2]*check[2]*koef, 2)
    elif score > 10 and productType == 'REFINANCE':
        return round(ARlist[int(round(score, -1)/10)-1][3]*check[3]*koef, 2)
    else:
        return round(ARlist[int(round(score, -1)/10)-1][0]*check[0]*koef, 2)

dr['createdAt'] = pd.to_datetime(dr['createdAt'])
# dr['payout'] = dr['payout'].astype(float)
# dr['payout'] = dr.apply(lambda row: 3800 if row['partner']=='Альфа-Банк' 
#                         else row['payout'], axis=1)
# dr['payout'] = dr.apply(lambda row: 2550 if row['partner']=='Kredito24' 
#                         else row['payout'], axis=1)
# dr['productType'] = dr['product'].apply(lambda x: x['productType'])


dr['source'] = dr.apply(lambda row: 'Металинвест Cash' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)

#dr = dr[dr['status_x']!='NEW']
dr['score'] = dr['score'].apply(lambda x: x+75 if x==0 else x)
dr['whence']=dr.apply(lambda row: row['whence_y'] if pd.notnull(row['whence_y']) else row['whence_x'],axis=1)
dr['source']=dr.apply(lambda row: row['utm_source_y'] if pd.notnull(row['utm_source_y']) else row['whence'],axis=1)
dr['medium']=dr.apply(lambda row: row['utm_medium_y'] if pd.notnull(row['utm_medium_y']) else row['whence'],axis=1)

#sms = dr.groupby('source').agg({'_id':'count'}).to_excel('sms.xlsx')

dr['source']=dr.apply(lambda row: row['utm_source_y'] if 
                      pd.notnull(row['utm_source_y']) else row['whence'],axis=1)
dr['source']=dr.apply(lambda row: row['sourceGlobal'] if 
                      pd.notnull(row['sourceGlobal']) else row['source'],axis=1)
dr['source']=dr.apply(lambda row: 'Город Тройка' if 
                      row['utm_content']=='gorodtroika' else row['source'],axis=1)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr'] else x)
dr['source'] = dr['source'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x in ['tele2TommorowWON',
                                         'tele2Tommorow',
                                         'tele2FourDays',
                                         'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
dr['source'] = dr['source'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
dr['source'] = dr['source'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
dr['source'] = dr['source'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click' 
                                  or x[:4]=='guru' or x[:9]=='guruleads' or x[:4]=='city' else x)
dr['source'] = dr['source'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
dr['source'] = dr['source'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele' else x)
dr['source'] = dr['source'].apply(lambda x: 'Tele2 // Кредитные карты' if 
                                   x[:5]=='t2_kk' else x)
dr['source'] = dr['source'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
dr['source'] = dr['source'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
dr['source'] = dr['source'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
dr['source'] = dr['source'].apply(lambda x: 'Ссылочная масса' if 
                                   x[:8]=='showcase' else x)
dr['source'] = dr['source'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content-text' else x)
dr['source'] = dr['source'].apply(lambda x: 'МТС \\ баннер + СМС' if 
                                   x[:2]=='mt' or x[:6]=='stream' else x)
dr['source'] = dr['source'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Крупный партнер' if 
#                                    x[:8]=='banki.ru' else x)            
dr['source'] = dr['source'].apply(lambda x: 'Тестирование гипотез' if 
                                  x[:6]=='yandex' or x[:6]=='google' else x)
dr['source']=dr.apply(lambda row: 'Тестирование гипотез' if 
                      row['utm_source_y']=='yandex' or row['utm_source_y']=='google' else row['source'],axis=1)
dr['source'] = dr.apply(lambda row: 'Крупный партнер (1)' if row['partner']=='banki.ru' 
                        else row['source'], axis=1)

dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)
dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5ef4473acf22a90001010445' 
                        else row['source'], axis=1)


def get_kibana_reg(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 1000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/1000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-1000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID','currentPage',
#                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
#                   'transaction_id', 'clickId', 
                'sourceGlobal']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

regs = get_kibana_reg(es, since,till)

# dr['utm_source_x'].value_counts()

regs['currentPage1'] = regs['currentPage'].apply(lambda x: x[:5] if pd.notnull(x) else np.nan)
regs['currentPage2'] = regs['currentPage'].apply(lambda x: x[:9] if pd.notnull(x) else np.nan)

dr=pd.merge(left=dr, right=regs, how='left', on='userId')

# odin = dr[dr['dt_reg'].notnull()]
# odin=odin[odin['product_type']=='LOAN']
# odin['userId'].nunique()

dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Android_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Huawei_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Apple_mobile' 
                        else row['source_x'], axis=1)

dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage1']=='/news' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage2']=='/articles' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Органический трафик' if row['source_x']=='Ссылочная масса' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'New (2Gis)' if row['source_x']=='2gis' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['source_x']=='Мобильное приложение' 
                        else row['source_x'], axis=1)
# Костыль

dr['status_x'] = dr.apply(lambda x: 'APPROVED' if
                        pd.notnull(x['status_y']) and x['status_y'] in(['APPROVED', 'approved', 'ISSUED']) 
                          else x['status_x'], axis=1)
                          


6441


INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:0.369s]


16381; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.181s]


6381; 

...

1981; 1981 raz


INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:0.033s]


1819; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.027s]


819; 

In [135]:
# dr[dr['partner']=='МТС Банк'].groupby(['product_name','status_x']).agg({'userId_x':'count'})

In [8]:
# dr[dr['product_type']=='DEBIT_CARD'].groupby(['week','type',
#                                                       'product_type','partner',
#                                                   'status_x']).agg({'userId_x':'count'})

In [9]:
# dr[dr['partner']=='Быстроденьги'].groupby('status_x').agg({'userId_x':'nunique','payout':'sum'})


,userId_x,payout
status_x,,


In [10]:
dr['dt'] = pd.to_datetime(dr['dt'])
dr['month'] = dr['dt'].apply(lambda x: datetime.strftime(x, '%Y-%m-%B'))


In [11]:
dr['week'] = (dr['dt'].dt.week)+1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


In [17]:
#dr[dr['type']=='API'].groupby(['month','type','partner','status_x']).agg({'userId_x':'count'}).to_excel('api.xlsx')

In [18]:
dr['Доход'] = dr.apply(lambda row: AR_count(row['score'], row['productType_x'], row['status_x']), axis=1)
# # Тут костыли для улучшения прогноза
dr['Доход'] = dr.apply(lambda row: row['Доход']/3 if row['type']=='API' else row['Доход'],axis=1)
dr['Доход'] = dr['Доход'].apply(lambda x: x*3)
dr['Доход'] = dr.apply(lambda row: row['Доход']/8 if row['source_x']=='exRejected' else row['Доход'],axis=1)
dr['Доход'] = dr.apply(lambda row: row['Доход'] if row['partner'] in ['еКапуста',
                                                                      'Деньги Сразу',
                                                                      'Веб-Займ']  
                       and row['status_x'] in ['APPROVED',
                                               'ISSUED'] else row['Доход'],axis=1)
dr['Доход'] = dr.apply(lambda row: row['Доход']/2 if row['source_x'] in ['Bankiros.ru ссылки']  and
                       row['status_x'] not in ['APPROVED',
                                               'ISSUED'] else row['Доход'],axis=1)
dr['Доход'] = dr.apply(lambda row: row['Доход']*9 if row['source_x'] in ['Tele2 // Микрозаймы']  and
                     row['status_x'] not in ['APPROVED',
                                               'ISSUED'] else row['Доход'],axis=1)
dr['Доход'] = dr.apply(lambda row: row['Доход']*6 if row['source_x'] in ['Vbr.ru ссылки']  and
                       row['status_x'] not in ['APPROVED',
                                             'ISSUED'] else row['Доход'],axis=1)
dr['Доход'] = dr.apply(lambda row: row['Доход']*2.5 if row['source_x'] in ['Email']  and
                     row['status_x'] not in ['APPROVED',
                                               'ISSUED'] else row['Доход'],axis=1)
dr['Доход'] = dr.apply(lambda row: row['Доход']*2 if row['source_x'] in ['SMS рекламные']  and
                     row['status_x'] not in ['APPROVED',
                                               'ISSUED'] else row['Доход'],axis=1)
# dr['Доход'] = dr.apply(lambda row: row['Доход']*2 if row['source_x'] in ['Мобильное приложение 1']  and
#                      row['status_x'] not in ['APPROVED',
#                                                'ISSUED'] else row['Доход'],axis=1)
dr['Доход'] = dr.apply(lambda row: row['Доход']*3 if row['source_x'] in ['Admitad']  and
                     row['status_x'] not in ['APPROVED',
                                               'ISSUED'] else row['Доход'],axis=1)
dr['Доход'] = dr.apply(lambda row: row['Доход']*2 if row['source_x'] in ['Adv.Cake']  and
                     row['status_x'] not in ['APPROVED',
                                               'ISSUED'] else row['Доход'],axis=1)
dr['Доход'] = dr.apply(lambda row: row['Доход']*3.3 if row['source_x'] in ['SMS рекламные']  and
                     row['status_x'] not in ['APPROVED',
                                               'ISSUED'] else row['Доход'],axis=1)
dr['Доход'] = dr.apply(lambda row: row['Доход']*8 if row['source_x'] in ['Крупный партнер (1)']  and
                     row['status_x'] in ['APPROVED',
                                               'ISSUED'] else row['Доход'],axis=1)
dr['Доход'] = dr.apply(lambda row: row['Доход']*3 if row['source_x'] in ['SMS рекламные']  and
                     row['status_x'] not in ['APPROVED',
                                               'ISSUED'] else row['Доход'],axis=1)


In [19]:
dr['payout'] = dr.apply(lambda row: row['payout']*1.7 if row['source_x'] in ['Крупный партнер (1)']  and
                     row['status_x'] in ['APPROVED',
                                               'ISSUED'] else row['payout'],axis=1)


In [20]:
dr['status_x'].unique()

array(['ACCEPTED', 'REJECTED', 'APPROVED'], dtype=object)

In [21]:
dr.groupby(['source_x']).agg({"_id": 'count',
                                  "Доход": 'sum'}).to_excel(f'reports/{datetime.strftime(since, "%B")}_channels.xlsx')
forecast = dr.groupby(['source_x']).agg({"status_x": 'count',
                                  "Доход": 'sum'}).sort_values('source_x',ascending=False)
forecast1 = dr.groupby(['source_x']).agg({"status_x": 'count',
                                  "payout": 'sum'}).sort_values('source_x',ascending=False)
                                                            
forecast2 = dr.groupby(['source_x','status_x'],as_index=False).agg({"userId_x": 'nunique',
                                  "payout": 'sum'})                                                           

In [22]:
forecast1

,status_x,payout
source_x,,
Статьи на сайте,6,0.0
Органический трафик,183,16780.0
Мобильное приложение 1,305,29324.0
Крупный партнер (1),83,0.0
Браузерные пуши (web),5,0.0
test_1488_344,1,3150.0
news,1,0.0
kupikupon,1,0.0
Vbr.ru ссылки,49,14070.0


In [23]:
forecast1['payout'].sum()

110194.0

In [24]:
# dr[dr['partner'].isin(['Металлинвестбанк',
#                        'Быстроденьги',
#                       'МигКредит'])].groupby(['partner','product_type','status_x']).agg({'_id': 'count',
#                                                                                          'payout':'sum'})

In [25]:
forecast2

,source_x,status_x,userId_x,payout
0,Admitad,ACCEPTED,47,0.0
1,Admitad,APPROVED,3,6275.0
2,Admitad,REJECTED,5,0.0
3,Adv.Cake,ACCEPTED,67,0.0
4,Adv.Cake,APPROVED,6,14808.0
5,Adv.Cake,REJECTED,19,0.0
6,Brobank.ru,ACCEPTED,11,0.0
7,Email,ACCEPTED,54,0.0
8,Email,APPROVED,7,15056.0
9,Email,REJECTED,15,0.0


In [26]:
forecast1

,status_x,payout
source_x,,
Статьи на сайте,6,0.0
Органический трафик,183,16780.0
Мобильное приложение 1,305,29324.0
Крупный партнер (1),83,0.0
Браузерные пуши (web),5,0.0
test_1488_344,1,3150.0
news,1,0.0
kupikupon,1,0.0
Vbr.ru ссылки,49,14070.0


In [27]:
forecast = forecast/(date.today().day-1)*monthrange(date.today().year, date.today().month)[1]

In [28]:
forecast

,status_x,Доход
source_x,,
Статьи на сайте,9.789474,520.212632
Органический трафик,298.578947,18274.695789
Мобильное приложение 1,497.631579,30056.131579
Крупный партнер (1),135.421053,9109.937368
Браузерные пуши (web),8.157895,344.654737
test_1488_344,1.631579,162.358421
news,1.631579,162.358421
kupikupon,1.631579,101.565789
Vbr.ru ссылки,79.947368,44486.207368


In [29]:
forecast['Доход'].sum()

289089.0919473684

In [30]:
forecast = forecast.astype('int').reset_index()
forecast.columns = ['Активность', 'Заявки факт', 'Выручка факт']
forecast['Расход факт'] = np.nan
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*0.95)*-1 if 
                       row['Активность'] in ['Adv.Cake', 'Город Тройка'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт'])*-1 if 
                       row['Активность'] in ['Vbr.ru ссылки', 'Bankiros.ru ссылки'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(24000)*-1 if 
                       row['Активность'] in ['Email', 'Браузерные пуши (web)'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*0.85)*-1 if 
                       row['Активность'] in ['Brobank.ru', 'Cpahub',
                                             'Exbico', 'Leadgid', 'Admitad',
                                             'Rubid', 'Перфоманс через Артикс'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*1.1)*-1 if 
                       row['Активность'] in ['SMS рекламные'] else row['Расход факт'],axis=1)

forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*0.75)*-1 if 
                       row['Активность'] in ['Tele2 // Микрозаймы', 'mega'] else row['Расход факт'],axis=1)
forecast['Расход факт']=forecast.apply(lambda row: round(row['Выручка факт']*0.85)*-1 if 
                       row['Активность'] in ['Металлинвест \\ потребы'] else row['Расход факт'],axis=1)

In [31]:
#!pip install google-auth-httplib2

In [32]:
# Пробуем получить данные по плану на 2021 год, а в случае неудачи 
try:

    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SPREADSHEET_ID = '1dFoqE0NuMX72hezUTByMnHUlW-YzyTM0xrFmvEBemes'
    RANGE_NAME = '2021_маркетинг (Общ)'
    data = pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME)
    df = pd.DataFrame(data[1:], columns=data[0])

    dates = []
    for i in range(14):
        d = date(2020, 11, 1)+relativedelta(months=i)
        d = datetime.strftime(d, "%Y, %B")
        dates.append(d)

    for old, new in zip(df.columns[7:21], dates):
        df[new] = df[old].apply(to_float)
# для текущей даты
    dp = pd.merge(left = df[(df['Прогноз']==' Выручка')&(df[datetime.strftime(datetime.now(), "%Y, %B")].notnull())
      &(~df['Активность'].isin(['', 'Общее'])&(df['Проект \ текущее']=='Текущее'))][['Активность', 
                                                  'Ответственный', datetime.strftime(datetime.now(), "%Y, %B")]],
            right = df[(df['Прогноз']==' Расход')&(df[datetime.strftime(datetime.now(), "%Y, %B")].notnull())
      &(~df['Активность'].isin(['', 'Общее'])&(df['Проект \ текущее']=='Текущее'))][['Активность', 
                                                 'Ответственный',datetime.strftime(datetime.now(), "%Y, %B")]],
            on = ['Ответственный','Активность'], suffixes=[' Выручка', ' Расход'], how = 'outer')

    dp = pd.merge(left = dp,
                right = df[(df['Прогноз']==' Заявки')&(df[datetime.strftime(datetime.now(), "%Y, %B")].notnull())
              &(~df['Активность'].isin(['', 'Общее'])&(df['Проект \ текущее']=='Текущее'))][['Активность', 
                                                 'Ответственный',datetime.strftime(datetime.now(), "%Y, %B")]],
            on = ['Ответственный','Активность'], how = 'outer').rename(columns = 
                                                        {datetime.strftime(datetime.now(), "%Y, %B"): 
                                                         datetime.strftime(datetime.now(), "%Y, %B")+' Заявки'})

    dp.to_pickle('reports/plan_2021.pickle')
    
except Exception as e:
    print('fail', e, e.args)
    dp = pd.read_pickle('reports/plan_2021.pickle')

COMPLETE: Data copied


In [33]:
#!pip install google-auth-httplib2

In [34]:
# Пробуем получить данные по плану на 2021 год, а в случае неудачи 
#try:

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '1PNnboEdVIn-MinWuXnH_GjubU4ti-Qh_OchHkH3lQSM'
RANGE_NAME = 'Актуализация_2кв'
data = pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME)
df1 = pd.DataFrame(data[1:], columns=data[0])

#     dates = []
#     for i in range(14):
#         d = date(2020, 11, 1)+relativedelta(months=i)
#         d = datetime.strftime(d, "%Y, %B")
#         dates.append(d)

#     for old, new in zip(df.columns[7:21], dates):
#         df[new] = df[old].apply(to_float)
# # для текущей даты
#     dp1 = pd.merge(left = df1[(df1['Прогноз']==' Выручка,нетт')&(df1[datetime.strftime(datetime.now(), "%Y, %B")].notnull())
#       &(~df1['Инициатива'].isin(['', 'Общее']))][['Инициатива', 
#                                                   'Отв', datetime.strftime(datetime.now(), "%Y, %B")]],
#             right = df1[(df1['Прогноз']==' Маркетинг')&(df[datetime.strftime(datetime.now(), "%Y, %B")].notnull())
#       &(~df['Инициатива'].isin(['', 'Общее']))][['Инициатива', 
#                                                  'Отв',datetime.strftime(datetime.now(), "%Y, %B")]],
#             on = ['Ответственный','Инициатива'], suffixes=[' Выручка', ' Расход'], how = 'outer')

#     dp1 = pd.merge(left = dp1,
#                 right = df1[(df1['Прогноз']==' Заявки')&(df1[datetime.strftime(datetime.now(), "%Y, %B")].notnull())
#               &(~df['Инициатива'].isin(['', 'Общее']))][['Инициатива', 
#                                                  'Отв',datetime.strftime(datetime.now(), "%Y, %B")]],
#             on = ['Ответственный','Инициатива'], how = 'outer').rename(columns = 
#                                                         {datetime.strftime(datetime.now(), "%Y, %B"): 
#                                                          datetime.strftime(datetime.now(), "%Y, %B")+' Заявки'})

#     dp.to_pickle('reports/plan_2021.pickle')
    
# except Exception as e:
#     print('fail', e, e.args)
#     dp = pd.read_pickle('reports/plan_2021.pickle')

COMPLETE: Data copied


In [35]:
df1 

,Показатель,Трафик,блок,Инициатива,Отв,план,план,план,план Апрель,план Май,план Июнь,план Июль,план Август,план Сентябрь,план Октябрь,план Ноябрь,план Декабрь,,факт,факт,прогноз,план Апрель акт,план Май акт,план Июнь акт,план Июль акт,план Август акт,план Сентябрь акт,план Октябрь акт,план Ноябрь акт,план Декабрь акт,,,,,ПЛАН,ПРОГНОЗ,откл,откл:%,ПЛАН,ПРОГНОЗ,ПЛАН,ПРОГНОЗ,ПЛАН,ПРОГНОЗ,ПЛАН,ПРОГНОЗ,откл,"откл,%"
0,,Статус,Блок,Инициатива,Отв,план Янв.,план Фев.,план Март,план Апрель,план Май,план Июнь,план Июль,план Авг,план Сент,план Окт,План Нояб,План Дек,Справочно,янв.1,февр.1,мар.1,апр.1,мая1,июн.1,июл.1,авг.1,сент.1,окт.1,нояб.1,дек.1,РАЗРАБОТКА,ПРИОРИТЕТ,,,1 КВАРТАЛ,,,,2 КВАРТАЛ,,3 КВАРТАЛ,,4 КВАРТАЛ,,Итого 2021 ГОД,None,None,None
1,Показатель,ИТОГО,ИТОГО,ИТОГО,ИТОГО,,,,,,,,,,,,,81 627,31 920,23 155,26 552,96 760,127 947,161 244,183 709,202 113,226 464,263 840,303 751,356 688,,,,,917 495,81 627,-835 868,-91%,1 372 962,385 951,2 022 727,612 286,924 279,924 279,5 237 463,2 004 143,-3 233 320,-62%
2,Трафик,Факт из Дэшборда,Direct,Email,Железнова Е,279 750,362 340,453 609,574 463,710 546,864 577,1 039 217,1 237 406,1 461 590,1 716 055,2 007 066,2 339 545,1 235 000,400 000,415 000,420 000,1 741 835,2 232 848,2 817 003,3 098 704,3 408 574,3 749 432,4 311 846,4 958 623,5 702 417,,,,,1 095 699,1 235 000,139 301,13%,2 149 585,6 791 686,3 738 213,10 256 709,6 062 667,14 972 886,13 046 164,33 256 282,20 210 117,155%
3,Трафик,Факт из Дэшборда,Direct,SMS рекламные,Железнова Е,101 911,123 885,136 905,181 280,204 125,231 046,261 960,297 283,336 276,381 698,436 516,498 718,13 500,4 500,4 500,4 500,703,901,1 136,1 250,1 375,1 513,1 739,2 000,2 300,,,,,362 701,13 500,-349 201,-96%,616 451,2 740,895 520,4 138,6 040,6 040,1 880 712,26 417,-1 854 294,-99%
4,Трафик,Факт из Дэшборда,Direct,Браузерные пуши (web),Железнова Е,45 134,54 459,63 786,78 664,93 572,107 282,122 460,139 300,155 886,174 890,198 833,224 204,1 104,737,315,52,,,,,,,,,,,,,,163 379,1 104,-162 275,-99%,279 518,0,417 647,0,0,0,860 544,1 104,-859 440,-100%
5,Трафик,Факт из Дэшборда,КЦ,Exbico,Дудченко С,50 000,55 000,60 500,66 550,73 205,80 526,88 578,97 436,107 179,117 897,129 687,142 656,239,119,70,50,,,,,,,,,,,,,,165 500,239,-165 261,-100%,220 281,0,293 193,0,0,0,678 974,239,-678 735,-100%
6,Трафик,Факт из Дэшборда,SEO,Наполнение лендингов по городам,Горяев К,0,11 111,15 873,22 222,31 111,43 556,60 978,85 369,89 637,94 119,119 516,125 492,0,,,,,,,,,,,,,,,,,26 984,0,-26 984,-100%,96 889,0,235 984,0,0,0,359 857,0,-359 857,-100%
7,Трафик,Факт из Дэшборда,SEO,Органический трафик,Горяев К,5 065,7 383,10 495,14 878,19 811,30 401,36 309,43 509,52 310,62 966,76 084,92 256,25 600,9 000,7 600,9 000,1 746,2 820,5 184,7 258,10 161,14 225,19 915,27 881,39 033,,,,,22 942,25 600,2 658,12%,65 090,9 750,132 128,31 643,86 829,86 829,306 989,153 822,-153 167,-50%
8,Трафик,Факт из Дэшборда,SEO,продвижение МП,Горяев К,0,0,0,0,0,0,0,0,0,0,0,0,1 025,170,230,625,704,787,882,970,1 067,1 174,1 350,1 553,1 785,,,,,0,1 025,1 025,0%,0,2 373,0,3 211,4 688,4 688,4 688,11 297,6 609,141%
9,Трафик,Факт из Дэшборда,CPA,Adv.Cake,Летников С,7 718,8 489,9 338,10 272,12 326,14 792,17 750,18 638,19 569,20 548,21 575,22 654,7 800,2 800,2 000,3 000,3 200,3 400,3 600,3 960,4 356,4 792,5 510,6 337,7 287,"new, да установка доп ПО для логирования Моб Прил",,,,25 545,7 800,-17 745,-69%,37 390,10 200,55 957,13 108,19 135,19 135,138 027,50 242,-87 784,-64%


In [36]:
month = 'Декабрь'

In [37]:
df2= df1[['Показатель','блок','Инициатива','Отв','план Декабрь','план Декабрь акт']]
df2= df2[df2['Показатель']=='Выручка, нетт']

df2=df2.rename(columns={"план Декабрь": "план Декабрь выручка",
                   "план Декабрь акт": "план Декабрь актуализация выручка"})
df2=df2[['блок','Инициатива','Отв','план Декабрь выручка','план Декабрь актуализация выручка']]

df3= df1[['Показатель','блок','Инициатива','Отв','план Декабрь','план Декабрь акт']]
df3= df3[df3['Показатель']=='Маркетинг, гросс']

df3=df3.rename(columns={"план Декабрь": "план Декабрь маркетинг",
                   "план Декабрь акт": "план Декабрь актуализация маркетинг"})
df3=df3[['блок','Инициатива','Отв','план Декабрь маркетинг','план Декабрь актуализация маркетинг']]

df4= df1[['Показатель','блок','Инициатива','Отв','план Декабрь','план Декабрь акт']]
df4= df4[df4['Показатель']=='Маржа, гросс']

df4=df4.rename(columns={"план Декабрь": "план Декабрь маржа",
                   "план Декабрь акт": "план Декабрь актуализация маржа"})
df4=df4[['блок','Инициатива','Отв','план Декабрь маржа','план Декабрь актуализация маржа']]

df5= df1[['Показатель','блок','Инициатива','Отв','план Декабрь','план Декабрь акт']]
df5= df5[df5['Показатель']=='Cost / Income']

df5=df5.rename(columns={"план Декабрь": "план Декабрь Cost / Income",
                   "план Декабрь акт": "план Декабрь актуализация Cost / Income"})
df5=df5[['блок','Инициатива','Отв','план Декабрь Cost / Income','план Декабрь актуализация Cost / Income']]

df6= df1[['Показатель','блок','Инициатива','Отв','план Декабрь','план Декабрь акт']]
df6= df6[df6['Показатель']=='Заявки']

df6=df6.rename(columns={"план Декабрь": "план Декабрь Заявки",
                   "план Декабрь акт": "план Декабрь актуализация Заявки"})
df6=df6[['блок','Инициатива','Отв','план Декабрь Заявки','план Декабрь актуализация Заявки']]

In [38]:
merge = df2.merge(df3,on=['Инициатива','Отв','блок'],how='left')

In [39]:
merge = merge.merge(df4,on=['Инициатива','Отв','блок'],how='left')
merge = merge.merge(df5,on=['Инициатива','Отв','блок'],how='left')
merge = merge.merge(df6,on=['Инициатива','Отв','блок'],how='left')

In [40]:
merge=merge.rename(columns={"Инициатива": "Активность",
                   "Отв": "Ответственный"})

In [41]:
merge =  merge.drop(index=[0])
merge

,блок,Активность,Ответственный,план Декабрь выручка,план Декабрь актуализация выручка,план Декабрь маркетинг,план Декабрь актуализация маркетинг,план Декабрь маржа,план Декабрь актуализация маржа,план Декабрь Cost / Income,план Декабрь актуализация Cost / Income,план Декабрь Заявки,план Декабрь актуализация Заявки
1,Direct,Email,Железнова Е,898 385,473 425,-40 000,-332 449,858 385,140 977,"-4,5%","-70,2%",9 358,2 170
2,Direct,SMS рекламные,Железнова Е,1 017 385,4 901,-947 564,-6 353,69 821,-1 452,"-93,1%","-129,6%",8 478,37
3,Direct,Браузерные пуши (web),Железнова Е,57 396,0,-30 000,0,27 396,0,"-52,3%",-,897,0
4,КЦ,Exbico,Дудченко С,1 355 230,0,-1 151 946,0,203 285,0,"-85,0%",-,14 266,0
5,SEO,Наполнение лендингов по городам,Горяев К,260 898,0,0,0,260 898,0,"0,0%",-,5 271,0
6,SEO,Органический трафик,Горяев К,191 801,61 039,0,-100 000,191 801,-38 961,"0,0%","-163,8%",3 875,556
7,SEO,продвижение МП,Горяев К,39 600,49 658,0,0,39 600,49 658,"0,0%","0,0%",800,497
8,CPA,Adv.Cake,Летников С,1 359 240,96 215,-1 087 392,-14 921,271 848,81 295,"-80,0%","-15,5%",3 398,1 179
9,CPA,Admitad,Летников С,171 716,21 249,-137 373,-72 619,34 343,-51 370,"-80,0%","-341,8%",572,841
10,CPA,Leadgid,Летников С,97 332,60 263,-77 866,-47 044,19 466,13 219,"-80,0%","-78,1%",324,504


In [42]:
forecast

,Активность,Заявки факт,Выручка факт,Расход факт
0,Статьи на сайте,9,520,NaN
1,Органический трафик,298,18274,NaN
2,Мобильное приложение 1,497,30056,NaN
3,Крупный партнер (1),135,9109,NaN
4,Браузерные пуши (web),8,344,-24000.0
5,test_1488_344,1,162,NaN
6,news,1,162,NaN
7,kupikupon,1,101,NaN
8,Vbr.ru ссылки,79,44486,-44486.0
9,Tele2 // Микрозаймы,84,70788,-53091.0


In [43]:
merge

,блок,Активность,Ответственный,план Декабрь выручка,план Декабрь актуализация выручка,план Декабрь маркетинг,план Декабрь актуализация маркетинг,план Декабрь маржа,план Декабрь актуализация маржа,план Декабрь Cost / Income,план Декабрь актуализация Cost / Income,план Декабрь Заявки,план Декабрь актуализация Заявки
1,Direct,Email,Железнова Е,898 385,473 425,-40 000,-332 449,858 385,140 977,"-4,5%","-70,2%",9 358,2 170
2,Direct,SMS рекламные,Железнова Е,1 017 385,4 901,-947 564,-6 353,69 821,-1 452,"-93,1%","-129,6%",8 478,37
3,Direct,Браузерные пуши (web),Железнова Е,57 396,0,-30 000,0,27 396,0,"-52,3%",-,897,0
4,КЦ,Exbico,Дудченко С,1 355 230,0,-1 151 946,0,203 285,0,"-85,0%",-,14 266,0
5,SEO,Наполнение лендингов по городам,Горяев К,260 898,0,0,0,260 898,0,"0,0%",-,5 271,0
6,SEO,Органический трафик,Горяев К,191 801,61 039,0,-100 000,191 801,-38 961,"0,0%","-163,8%",3 875,556
7,SEO,продвижение МП,Горяев К,39 600,49 658,0,0,39 600,49 658,"0,0%","0,0%",800,497
8,CPA,Adv.Cake,Летников С,1 359 240,96 215,-1 087 392,-14 921,271 848,81 295,"-80,0%","-15,5%",3 398,1 179
9,CPA,Admitad,Летников С,171 716,21 249,-137 373,-72 619,34 343,-51 370,"-80,0%","-341,8%",572,841
10,CPA,Leadgid,Летников С,97 332,60 263,-77 866,-47 044,19 466,13 219,"-80,0%","-78,1%",324,504


In [44]:
#!pip install  google-auth-httplib2 

In [45]:
result = pd.merge(merge,forecast,on='Активность',how='left')
result

,блок,Активность,Ответственный,план Декабрь выручка,план Декабрь актуализация выручка,план Декабрь маркетинг,план Декабрь актуализация маркетинг,план Декабрь маржа,план Декабрь актуализация маржа,план Декабрь Cost / Income,план Декабрь актуализация Cost / Income,план Декабрь Заявки,план Декабрь актуализация Заявки,Заявки факт,Выручка факт,Расход факт
0,Direct,Email,Железнова Е,898 385,473 425,-40 000,-332 449,858 385,140 977,"-4,5%","-70,2%",9 358,2 170,145.0,27703.0,-24000.0
1,Direct,SMS рекламные,Железнова Е,1 017 385,4 901,-947 564,-6 353,69 821,-1 452,"-93,1%","-129,6%",8 478,37,35.0,47763.0,-52539.0
2,Direct,Браузерные пуши (web),Железнова Е,57 396,0,-30 000,0,27 396,0,"-52,3%",-,897,0,8.0,344.0,-24000.0
3,КЦ,Exbico,Дудченко С,1 355 230,0,-1 151 946,0,203 285,0,"-85,0%",-,14 266,0,NaN,NaN,NaN
4,SEO,Наполнение лендингов по городам,Горяев К,260 898,0,0,0,260 898,0,"0,0%",-,5 271,0,NaN,NaN,NaN
5,SEO,Органический трафик,Горяев К,191 801,61 039,0,-100 000,191 801,-38 961,"0,0%","-163,8%",3 875,556,298.0,18274.0,NaN
6,SEO,продвижение МП,Горяев К,39 600,49 658,0,0,39 600,49 658,"0,0%","0,0%",800,497,NaN,NaN,NaN
7,CPA,Adv.Cake,Летников С,1 359 240,96 215,-1 087 392,-14 921,271 848,81 295,"-80,0%","-15,5%",3 398,1 179,172.0,18849.0,-17907.0
8,CPA,Admitad,Летников С,171 716,21 249,-137 373,-72 619,34 343,-51 370,"-80,0%","-341,8%",572,841,99.0,18815.0,-15993.0
9,CPA,Leadgid,Летников С,97 332,60 263,-77 866,-47 044,19 466,13 219,"-80,0%","-78,1%",324,504,NaN,NaN,NaN


In [46]:
result = result[['блок','Активность', 'Ответственный',  'план Декабрь выручка', 'план Декабрь актуализация выручка',
       'Выручка факт', 'план Декабрь маркетинг', 'план Декабрь актуализация маркетинг', 'Расход факт',
        'план Декабрь Заявки','план Декабрь актуализация Заявки','Заявки факт'
       ]]
result

,блок,Активность,Ответственный,план Декабрь выручка,план Декабрь актуализация выручка,Выручка факт,план Декабрь маркетинг,план Декабрь актуализация маркетинг,Расход факт,план Декабрь Заявки,план Декабрь актуализация Заявки,Заявки факт
0,Direct,Email,Железнова Е,898 385,473 425,27703.0,-40 000,-332 449,-24000.0,9 358,2 170,145.0
1,Direct,SMS рекламные,Железнова Е,1 017 385,4 901,47763.0,-947 564,-6 353,-52539.0,8 478,37,35.0
2,Direct,Браузерные пуши (web),Железнова Е,57 396,0,344.0,-30 000,0,-24000.0,897,0,8.0
3,КЦ,Exbico,Дудченко С,1 355 230,0,NaN,-1 151 946,0,NaN,14 266,0,NaN
4,SEO,Наполнение лендингов по городам,Горяев К,260 898,0,NaN,0,0,NaN,5 271,0,NaN
5,SEO,Органический трафик,Горяев К,191 801,61 039,18274.0,0,-100 000,NaN,3 875,556,298.0
6,SEO,продвижение МП,Горяев К,39 600,49 658,NaN,0,0,NaN,800,497,NaN
7,CPA,Adv.Cake,Летников С,1 359 240,96 215,18849.0,-1 087 392,-14 921,-17907.0,3 398,1 179,172.0
8,CPA,Admitad,Летников С,171 716,21 249,18815.0,-137 373,-72 619,-15993.0,572,841,99.0
9,CPA,Leadgid,Летников С,97 332,60 263,NaN,-77 866,-47 044,NaN,324,504,NaN


In [47]:
result['Ответственный'] = result.apply(lambda row: 'Горяев К' if row['Активность']=='Мобильное приложение 1' 
                        else row['Ответственный'], axis=1)
result['Ответственный'] = result.apply(lambda row: 'Глущенко Д' if row['Активность']=='CallTat' 
                        else row['Ответственный'], axis=1)
result['Ответственный'] = result.apply(lambda row: 'Глущенко Д' if row['Активность']=='Пуш партнерка' 
                        else row['Ответственный'], axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [48]:
result = result.rename(columns={"Выручка факт": "Выручка прогноз",
                   "Расход факт": "Расход прогноз",
                   "Заявки факт": "Заявки прогноз"})

In [49]:
result

,блок,Активность,Ответственный,план Декабрь выручка,план Декабрь актуализация выручка,Выручка прогноз,план Декабрь маркетинг,план Декабрь актуализация маркетинг,Расход прогноз,план Декабрь Заявки,план Декабрь актуализация Заявки,Заявки прогноз
0,Direct,Email,Железнова Е,898 385,473 425,27703.0,-40 000,-332 449,-24000.0,9 358,2 170,145.0
1,Direct,SMS рекламные,Железнова Е,1 017 385,4 901,47763.0,-947 564,-6 353,-52539.0,8 478,37,35.0
2,Direct,Браузерные пуши (web),Железнова Е,57 396,0,344.0,-30 000,0,-24000.0,897,0,8.0
3,КЦ,Exbico,Дудченко С,1 355 230,0,NaN,-1 151 946,0,NaN,14 266,0,NaN
4,SEO,Наполнение лендингов по городам,Горяев К,260 898,0,NaN,0,0,NaN,5 271,0,NaN
5,SEO,Органический трафик,Горяев К,191 801,61 039,18274.0,0,-100 000,NaN,3 875,556,298.0
6,SEO,продвижение МП,Горяев К,39 600,49 658,NaN,0,0,NaN,800,497,NaN
7,CPA,Adv.Cake,Летников С,1 359 240,96 215,18849.0,-1 087 392,-14 921,-17907.0,3 398,1 179,172.0
8,CPA,Admitad,Летников С,171 716,21 249,18815.0,-137 373,-72 619,-15993.0,572,841,99.0
9,CPA,Leadgid,Летников С,97 332,60 263,NaN,-77 866,-47 044,NaN,324,504,NaN


In [50]:
result.at[0,'план Деквбрь актуализация выручка']=74511
result.at[1,'план Декбрь актуализация выручка']=17853
result.at[5,'план Декабрь актуализация выручка']=25000
result.at[7,'план Декабрь актуализация выручка']=20400
result.at[8,'план Декабрь актуализация выручка']=16000
result.at[9,'план Декабрь актуализация выручка']=2500
result.at[10,'план Декабрь актуализация выручка']=66000
result.at[19,'план Декабрь актуализация выручка']=152350
result.at[22,'план Декабрь актуализация выручка']=4600
result.at[56,'план Декабрь актуализация выручка']=0
result.at[57,'план Декабрь актуализация выручка']=0
result.at[51,'план Декабрь актуализация выручка']=0
result.at[52,'план Декабрь актуализация выручка']=0
result.at[38,'план Декабрь актуализация выручка']=0
result.at[33,'план Декабрь актуализация выручка']=0
result.at[16,'план Декабрь актуализация выручка']=0
result.at[15,'план Декабрь актуализация выручка']=500
result.at[11,'план Декабрь актуализация выручка']=0
result.at[16,'план Декабрь актуализация выручка']=0
result.at[6,'план Декабрь актуализация выручка']=0
result.at[17,'план Декабрь актуализация выручка']=0
result.at[23,'план Декабрь актуализация выручка']=18000
result.at[26,'план Декабрь актуализация выручка']=0
result.at[27,'план Декабрь актуализация выручка']=0
result.at[28,'план Декабрь актуализация выручка']=0
result.at[31,'план Декабрь актуализация выручка']=0
result.at[16,'план Декабрь актуализация выручка']=0
result.at[32,'план Декабрь актуализация выручка']=0
result.at[40,'план Декабрь актуализация выручка']=0
result.at[43,'план Декабрь актуализация выручка']=0
result.at[44,'план Декабрь актуализация выручка']=0
result.at[54,'план Декабрь актуализация выручка']=0
result.at[55,'план Декабрь актуализация выручка']=0
result.at[58,'план Декабрь актуализация выручка']=0
result.at[53,'план Декабрь актуализация выручка']=0
result.at[16,'план Декабрь актуализация выручка']=0
result.at[46,'план Декабрь актуализация выручка']=31317
result.at[56,'план Декабрь актуализация выручка']=60000

In [51]:
result

,блок,Активность,Ответственный,план Декабрь выручка,план Декабрь актуализация выручка,Выручка прогноз,план Декабрь маркетинг,план Декабрь актуализация маркетинг,Расход прогноз,план Декабрь Заявки,план Декабрь актуализация Заявки,Заявки прогноз,план Деквбрь актуализация выручка,план Декбрь актуализация выручка
0,Direct,Email,Железнова Е,898 385,473 425,27703.0,-40 000,-332 449,-24000.0,9 358,2 170,145.0,74511.0,NaN
1,Direct,SMS рекламные,Железнова Е,1 017 385,4 901,47763.0,-947 564,-6 353,-52539.0,8 478,37,35.0,NaN,17853.0
2,Direct,Браузерные пуши (web),Железнова Е,57 396,0,344.0,-30 000,0,-24000.0,897,0,8.0,NaN,NaN
3,КЦ,Exbico,Дудченко С,1 355 230,0,NaN,-1 151 946,0,NaN,14 266,0,NaN,NaN,NaN
4,SEO,Наполнение лендингов по городам,Горяев К,260 898,0,NaN,0,0,NaN,5 271,0,NaN,NaN,NaN
5,SEO,Органический трафик,Горяев К,191 801,25000,18274.0,0,-100 000,NaN,3 875,556,298.0,NaN,NaN
6,SEO,продвижение МП,Горяев К,39 600,0,NaN,0,0,NaN,800,497,NaN,NaN,NaN
7,CPA,Adv.Cake,Летников С,1 359 240,20400,18849.0,-1 087 392,-14 921,-17907.0,3 398,1 179,172.0,NaN,NaN
8,CPA,Admitad,Летников С,171 716,16000,18815.0,-137 373,-72 619,-15993.0,572,841,99.0,NaN,NaN
9,CPA,Leadgid,Летников С,97 332,2500,NaN,-77 866,-47 044,NaN,324,504,NaN,NaN,NaN


In [52]:
result.to_excel(f'reports/December_forecast.xlsx')

In [53]:
#!pip install --upgrade tapi-yandex-metrika==2020.10.20

In [54]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           # 'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][4]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'gender':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df = df.groupby(['week'],as_index=False).agg({'visits':'sum'})
df.to_excel('Еженедельная презентация/траффик_сайт.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(0) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Еженедельная презентация/траффик_сайт.xlsx'
                  )
set_with_dataframe(worksheet, df)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [55]:
df

,Unnamed: 0,week,visits
0,0,2,24609
1,1,3,24079
2,2,4,23627
3,3,5,22498
4,4,6,23901
5,5,7,23319
6,6,8,22845
7,7,9,19388
8,8,10,20275
9,9,11,15506


In [56]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:searchEngine",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
    #filters = "ym:s:lastTrafficSource=='organic'"
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][1]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'gender':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df = df.groupby(['week','utm_source'],as_index=False).agg({'visits':'sum'})
df = df.groupby(['week'],as_index=False).agg({'visits':'sum'})

df.to_excel('Еженедельная презентация/траффик_seo.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(1) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df = pd.read_excel('Еженедельная презентация/траффик_seo.xlsx'
                  )
set_with_dataframe(worksheet, df)


INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [58]:
since = date(2021, 1, 4)
till = date(2021, 12,31)

df = get_exact_claims(since, till, client)
df['requestId'] = df[df['partner']=='MoneyMan']['statusFields'].apply(mm_requestId)
dpbk = get_kibana_postback(es, since)
def float_apply(x):
    try:
        return float(x)
    except ValueError:
        return None
dpbk['payout'] = dpbk['payout'].apply(float_apply)
dpbk['payout'] = dpbk.apply(lambda x: x['payout']*1.2 if x['partnerName']=='admitad' else x['payout'], axis=1)
dpbk = dpbk[dpbk['status'].isin(['APPROVED', 'approved', 'ISSUED'])].sort_values('payout').drop_duplicates('transitionId', keep='last')
dr = pd.merge(left=df, right=dpbk, left_on='id', right_on='transitionId', how='left')
dc = get_kibana_claims(es, since,till)[['clickId', 'userId', 'utm_campaign', 'utm_medium', 'utm_content', 
                                        'utm_source', 'utm_term', 'gaId', 'yandexClientID', 'sourceGlobal']]
claims = pd.merge(left=dr, right=dc[dc['clickId'].notnull()].drop_duplicates('clickId'), on='clickId', how='left')

38036


INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:7.382s]


572024; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.920s]


562024; 

...

25229; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.838s]


15229; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.282s]


5229; 

In [60]:
claims=claims[claims['status_x']!='NEW']

In [61]:
claims['dt'] = pd.to_datetime(claims['dt'])
claims['week'] = claims['dt'].dt.week
claims['week'] = claims['week']+1
week_claim = claims.groupby(['week'],as_index=False).agg({'userId_x':'count'})
week_claim.to_excel('Еженедельная презентация/claim_week.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(2) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/claim_week.xlsx')
set_with_dataframe(worksheet, df1)   

week_claim_prod = claims.groupby(['week','product_type'],as_index=False).agg({'userId_x':'count'})
week_claim_prod.to_excel('Еженедельная презентация/claim_week_prod.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(3) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/claim_week_prod.xlsx')
set_with_dataframe(worksheet, df1)   

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [62]:
def get_kibana_gate(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"TildaForm"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}},
"_source":["timestamp",'eventType', "params.yandexClientID", "params.userId", "params.userId","params.TildaForm.utm_source",
           "params.TildaForm.utm_medium","params.TildaForm.cookies","params.TildaForm.phone","params.TildaForm.email",
           "params.currentPage", "params.gaId"]}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.2)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-5000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    #dr = dr[dr['params'].notnull()]
    dr.params.apply(lambda x: x.keys())
    for col in ['TildaForm'
            ]:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)
    dr.TildaForm.apply(lambda x: x.keys())
    for col in ['utm_medium','utm_source','cookies','phone','email'
            ]:
        dr[col] = dr['TildaForm'].apply(lambda x: x[col] if col in x.keys() else np.nan)
    del dr['params']
    del dr['TildaForm']

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

In [63]:
def get_kibana_click_broker(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(date.today()+timedelta(days=1), datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"CLAIM_BROKER"}},
#                          {"match_phrase":{"params.whence.keyword":{"query": whence }}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}
             # ,"_source": ["_id","params.clickId", "params.pid", "params.reqid", "params.wid","timestamp"]
              }
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['_id','pid','clickId','wid','reqId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

dobs= get_kibana_click_broker(es,since,till)

dobs = dobs[['clickId','pid','reqId','date']]
dobs= dobs[~dobs['clickId'].isin(['gtm-msr.appspot.com','test'])]
dobs['week']=(pd.to_datetime(dobs['date']).dt.week)+1
dobs

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:0.201s]


968; 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


,clickId,pid,reqId,date,week
119,1452323,,7080597,2021-11-17,47
120,android,app,7094001,2021-11-21,47
121,9m7Z1VjhNmZ9IPD,cityads,7128421,2021-11-29,49
122,9m7Z1VjFN3ZEDEw,cityads,7132673,2021-11-30,49
123,9m7Z1VjKlvZiBAG,cityads,7133644,2021-11-30,49
124,9m7Z1VjTUKZ9Mpg,cityads,7133644,2021-12-01,49
125,9m7Z1Vk2o6Zn2mx,cityads,7136531,2021-12-01,49
126,9m7Z1ViYPnZebMj,cityads,7125135,2021-11-28,48
127,9m7Z1Vjdw0ZA84N,cityads,7127095,2021-11-29,49
128,9m7Z1VjF0sZiC1G,cityads,7132495,2021-11-30,49


In [64]:
dg = get_kibana_gate(es, since,till)
del dg['timestamp']
#group = dg.pivot_table(index='utm_source',columns='dt',values='utm_medium',aggfunc=('count'))
dg['dt']= pd.to_datetime(dg['dt'])
dg['week']=dg['dt'].dt.week
dg['week']=dg['week']+1
dg['test'] = 1
gd = dg.drop_duplicates('phone')
group = dg.pivot_table(index='week',columns='utm_source',values='utm_medium',aggfunc=('count'))
group

group.to_excel('Еженедельная презентация/broker_channel.xlsx')
# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(6) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/broker_channel.xlsx')
set_with_dataframe(worksheet, df1)      

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:32.605s]


1428; 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


In [65]:
dg= dg.drop_duplicates('phone')
group = dg.groupby(['week'],as_index=False).agg({'utm_source':'count'})


In [66]:
dobs_week=dobs.groupby('week',as_index=False).agg({'clickId':'count'})
dobs_week.columns=['week','utm_source']
dobs_week

,week,utm_source
0,47,7
1,48,20
2,49,73
3,50,115
4,51,83


In [67]:
group2= pd.concat([group,dobs_week])
group3 = group2.groupby(['week'],as_index=False).agg({'utm_source':'sum'})


In [68]:
#group1 = dg.groupby('week',as_index=False).agg({'utm_source':'count'})
#group3['utm_source']=group1['utm_source'].astype('int')

group3.to_excel('Еженедельная презентация/broker_claim.xlsx')
# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(7) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/broker_claim.xlsx')
set_with_dataframe(worksheet, df1) 

In [69]:
dobs['pid']= dobs['pid'].fillna('Unnamed')
dobs.groupby(['week','pid'],as_index=False).agg({'date':'count'}).to_excel('Еженедельная презентация/dobs_claim.xlsx')
# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(16) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/dobs_claim.xlsx')
set_with_dataframe(worksheet, df1) 

In [70]:
def get_kibana_onboard(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 5000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"ONBOARDING"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}},
"_source":["timestamp", "params.yandexClientID", "params.userId", "params.userId",
           "params.currentPage", "params.gaId","params.sourceGlobal"]}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.2)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/5000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-5000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr = dr[dr['params'].notnull()]
#     dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId','sourceGlobal'
                ]:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)
    del dr['params']

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

In [71]:
onboard = get_kibana_onboard(es, since,till)

onboard['dt']=pd.to_datetime(onboard['dt'])
onboard['week'] = (onboard['dt'].dt.week)+1
onboard_group = onboard.groupby(['sourceGlobal','week','userId'],as_index=False).agg({'timestamp':'count'})
onboard_group2 = onboard.groupby(['sourceGlobal','week'],as_index=False).agg({'timestamp':'count'})

onboard_group2.to_excel('Еженедельная презентация/Инсталы_приложение.xlsx')
# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(8) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/Инсталы_приложение.xlsx')
set_with_dataframe(worksheet, df1) 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:1.773s]


17845; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.651s]


12845; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.377s]


7845; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.420s]


2845; 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.


In [72]:

def get_kibana_reg(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 1000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/1000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-1000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID','currentPage',
#                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
#                   'transaction_id', 'clickId', 
                'sourceGlobal']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr


In [73]:
reg = get_kibana_reg(es, since,till)

reg = reg[reg['sourceGlobal'].isin(['Android_mobile','Huawei_mobile','Apple_mobile'])]
reg['dt']=reg['dt_reg']
reg['dt']=pd.to_datetime(reg['dt'])
reg['week'] = (reg['dt'].dt.week)+1
dg_reg_mobile = reg.groupby(['week','sourceGlobal','userId'], as_index=False).agg({'dt':'count'})
dg_reg_mobile.to_excel('Еженедельная презентация/регистрации по мп.xlsx')
# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(22) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/регистрации по мп.xlsx')
set_with_dataframe(worksheet, df1)

reg = get_kibana_reg(es, since,till)
reg = reg[reg['sourceGlobal'].isin(['Android_mobile','Huawei_mobile','Apple_mobile'])]
reg['dt']=reg['dt_reg']
reg['dt']=pd.to_datetime(reg['dt'])
reg['week'] = (reg['dt'].dt.week)+1
dg1 = reg.groupby(['week','sourceGlobal','productType','userId'], as_index=False).agg({'dt':'count'})


onboard1 = onboard_group.merge(dg1,how='left',on=(['sourceGlobal','week','userId']))

onboard1 = onboard1.groupby(['sourceGlobal','week']).agg({'timestamp':'sum','dt':'sum'}).reset_index()


onboard1['CR_reg']= ((onboard1['dt']/onboard1['timestamp'])*100).round(2)

onboard1['CR_reg'].mean()

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:5.254s]


35454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:2.906s]


34454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:1.768s]


...

1454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.109s]


454; 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


35.84365217391305

In [75]:
# df = claims
# df = df.rename(columns={
#                         "product_type": "productType"})
# df['productType'] = df.apply(lambda row: 'card' if row['productType']=='CREDIT_CARD' else row['productType'], axis=1)
# df['productType'] = df.apply(lambda row: 'cash' if row['productType']=='CASH_LOAN' 
#                         else row['productType'], axis=1)
# df['productType'] = df.apply(lambda row: 'loan' if row['productType']=='LOAN' 
#                         else row['productType'], axis=1)
# df['productType'] = df.apply(lambda row: 'refinance' if row['productType']=='REFINANCE' 
#                         else row['productType'], axis=1)
# df= df.groupby(['week','productType','userId'], as_index=False).agg({'productId':'count'})
# df['unique']=1
# onboard_group1 = dg1
# onboard_group1 = onboard_group1.merge(df,how='left',on=(['userId','week']))
# onboard_group1['cr_reg_claim'] = ((onboard_group1['productId']/onboard_group1['dt'])*100).round(2)
# onboard_group1['cr_reg_claim_u'] = ((onboard_group1['unique']/onboard_group1['dt'])*100).round(2)


In [76]:
# final = onboard_group1.groupby(['week','sourceGlobal'],as_index=False).agg({'dt':'sum',
#                                                                                             'productId':'sum',
#                                                                                             })
# # final['cr_reg_claim'] = ((final['productId']/final['dt'])*100).round(2)
# # final['cr_reg_claim_u'] = ((final['unique']/final['dt'])*100).round(2)
# final.to_excel('Еженедельная презентация/Воронка по мобильному приложению.xlsx')# Доступ к GS
# final

In [77]:

# gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
# sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
# worksheet = sh.get_worksheet(10) #-> 0 - лист один, 1 - второй и тд.

# # Выбор датафрейма
# df1 = pd.read_excel('Еженедельная презентация/Воронка по мобильному приложению.xlsx')
# set_with_dataframe(worksheet, df1) 

In [78]:
def get_kibana_screensplash(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 5000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"SPLASH_SCREEN"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}},
"_source":["timestamp", "params.yandexClientID", "params.userId", "params.userId",
           "params.currentPage", "params.gaId","params.sourceGlobal"]}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.2)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/5000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-5000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr = dr[dr['params'].notnull()]
#     dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId','utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'currentPage',
                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', 'payout', 'status',
                   'transaction_id', 'clickId', 'sourceGlobal'
                ]:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)
    del dr['params']

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

In [79]:
screen_splash = get_kibana_screensplash(es, since,till)

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:9.375s]


37674; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:3.434s]


32674; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:1.124s]


...

12674; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:7.464s]


7674; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.578s]


2674; 

In [120]:
screen_splash['dt']=pd.to_datetime(screen_splash['dt'])
screen_splash['week'] = (screen_splash['dt'].dt.week)+1
dg1 = screen_splash.groupby(['week'], as_index=False).agg({'dt':'count'})
dg1.to_excel('Еженедельная презентация/Заходы по мобильному приложению.xlsx')
# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(10) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/Заходы по мобильному приложению.xlsx')
set_with_dataframe(worksheet, df1) 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [82]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    filters = "ym:s:UTMSource!='tele2'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           # 'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][4]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'gender':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df = df.groupby(['week'],as_index=False).agg({'visits':'sum'})
reg = get_kibana_reg(es, since,till)

reg=reg[~reg['sourceGlobal'].isin(['Android_mobile','Huawei_mobile','Apple_mobile'])]
reg['dt']=reg['dt_reg']
reg['dt']=pd.to_datetime(reg['dt'])
reg['week'] = (reg['dt'].dt.week)+1
reg = reg.groupby(['week'],as_index = False).agg({'dt':'count'})
df = df.merge(reg,how='left',on='week')
df['cr'] = ((df['dt']/df['visits'])*100).round(2)
df.to_excel('Еженедельная презентация/трафик в регу сайт.xlsx')
# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(11) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/трафик в регу сайт.xlsx')
set_with_dataframe(worksheet, df1) 

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:0.041s]


35454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.033s]


34454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.034s]


...

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.041s]


1454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.022s]


454; 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:63: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [83]:
reg = get_kibana_reg(es, since,till)

reg=reg[~reg['sourceGlobal'].isin(['Android_mobile','Huawei_mobile','Apple_mobile'])]
reg['dt'] = reg['dt_reg']
reg['dt']=pd.to_datetime(reg['dt'])
reg['week'] = (reg['dt'].dt.week)+1
dg1 = reg.groupby(['week','productType','userId'], as_index=False).agg({'dt':'count'})
dg2 = reg[reg['productType']=='refinance'].groupby(['week','sourceGlobal'], as_index=False).agg({'dt':'count'})

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:0.179s]


35454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.053s]


34454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.187s]


...

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.047s]


1454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.021s]


454; 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [84]:
df = claims
df = df[df['status_x']!='NEW']
df['dt']=pd.to_datetime(df['dt'])
df['week'] = (df['dt'].dt.week)+1
df['product_type']= df['product_type'].str.lower()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.


In [85]:
dg1 = dg1.rename(columns={
                        "userId": "userId_x"})

In [86]:
df= df.groupby(['week','product_type','userId_x'], as_index=False).agg({'productId_x':'count'})
df['unique']=1
df = df.rename(columns={
                        "product_type": "productType"})
dg = dg1.merge(df,how='left',on=(['week','userId_x']))
final = dg.groupby(['week','productType_x'],as_index=False).agg({'dt':'sum','productId_x':'sum','unique':'sum'})
final['cr_reg_claim'] = ((final['productId_x']/final['dt'])*100).round(2)
final['cr_reg_claim'] = ((final['productId_x']/final['dt'])*100).round(2)
final['cr_reg_claim_u'] = ((final['unique']/final['dt'])*100).round(2)
final.to_excel('Еженедельная презентация/CR без мобилки сайт.xlsx')
# Доступ к GS
import gspread
from gspread_dataframe import set_with_dataframe
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(12) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/CR без мобилки сайт.xlsx')
set_with_dataframe(worksheet, df1) 

In [87]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource",
    date1 ="2021-01-04",
    date2 = "yesterday",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
    #filters = "ym:s:UTMSource!='tele2'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][4]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'gender':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df = df.groupby(['week','utm_source'],as_index=False).agg({'visits':'sum'})
reg = get_kibana_reg(es, since,till)

reg=reg[~reg['sourceGlobal'].isin(['Android_mobile','Huawei_mobile','Apple_mobile'])]
reg['dt']=reg['dt_reg']
reg['dt']=pd.to_datetime(reg['dt'])
reg['week'] = (reg['dt'].dt.week)+1
reg = reg.groupby(['week','utm_source'],as_index = False).agg({'dt':'count'})
df = df.merge(reg,how='left',on=['week','utm_source'])
df['cr'] = ((df['dt']/df['visits'])*100).round(2)
df = df.fillna(0)
df =df[df['visits']> 10]
df.to_excel('Еженедельная презентация/трафик в регу сайт по UTM.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe
# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(15) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/трафик в регу сайт по UTM.xlsx')
set_with_dataframe(worksheet, df1) 

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:0.038s]


35454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.033s]


34454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.035s]


...

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.041s]


1454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.023s]


454; 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:63: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [88]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][4]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'gender':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df_utm = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df_utm['date'] = pd.to_datetime(df_utm['date'])
df_utm['week'] =(df_utm['date'].dt.week)+1

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [89]:
df_utm  = df_utm.groupby(['week','utm_source'],as_index=False).agg({'visits':'sum'})
df_utm = df_utm[df_utm['utm_source'].notnull()]

In [90]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:searchEngine,ym:s:lastTrafficSource",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
    #filters = "ym:s:lastTrafficSource=='organic'"
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
             'visit-id':result[i]["dimensions"][2]["name"],
            #'area':result[i]["dimensions"][1]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'gender':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df = df.groupby(['week','utm_source','visit-id'],as_index=False).agg({'visits':'sum'})
df = df.groupby(['week','utm_source','visit-id']).agg({'visits':'sum'})

# df.to_excel('Еженедельная презентация/траффик_seo.xlsx')
# import gspread
# from gspread_dataframe import set_with_dataframe

# # Доступ к GS
# gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
# sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
# worksheet = sh.get_worksheet(1) #-> 0 - лист один, 1 - второй и тд.

# # Выбор датафрейма
# df = pd.read_excel('Еженедельная презентация/траффик_seo.xlsx'
#                   )
# set_with_dataframe(worksheet, df)
df

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


visits
week utm_source              visit-id                     
2    Google, search results  Search engine traffic  1994.0
     Google: mobile app      Search engine traffic     8.0
     Mail.ru, search results Search engine traffic     6.0
     Yahoo, search results   Search engine traffic     1.0
     Yandex Mobile           Search engine traffic    44.0
     Yandex, search results  Search engine traffic    59.0
     Yandex: mobile app      Search engine traffic    18.0
3    Baidu, search results   Search engine traffic     1.0
     Google, search results  Search engine traffic  2256.0
     Google: mobile app      Search engine traffic     7.0
     Mail.ru, search results Search engine traffic     7.0
     Yandex Mobile           Search engine traffic    33.0
     Yandex, search results  Search engine traffic    96.0
     Yandex.Images           Search engine traffic     2.0
     Yandex: mobile app      Search engine traffic    21.0
4    DuckDuckGo              Search engine traffic     2.0
     Google, search results  Search engine traffic  2112.0
     Google: mobile app      Search engine traffic    11.0
     Mail.ru, search results Search engine traffic    15.0
     Ukr.net - Other         Search engine traffic     1.0
     Yahoo, search results   Search engine traffic     1.0
     Yandex Mobile           Search engine traffic    64.0
     Yandex, search results  Search engine traffic   139.0
     Yandex: mobile app      Search engine traffic    24.0
5    Baidu, search results   Search engine traffic     2.0
     Bing, search results    Search engine traffic     1.0
     Ecosia                  Search engine traffic     1.0
     Google, search results  Search engine traffic  1676.0
     Google: mobile app      Search engine traffic     3.0
     Mail.ru, search results Search engine traffic     6.0
     Yahoo, search results   Search engine traffic     1.0
     Yandex Mobile           Search engine traffic    68.0
     Yandex, search results  Search engine traffic   166.0
     Yandex.Images           Search engine traffic     2.0
     Yandex: mobile app      Search engine traffic    19.0
6    Baidu, search results   Search engine traffic     3.0
     Google, search results  Search engine traffic  1458.0
     Google: mobile app      Search engine traffic     5.0
     Mail.ru, search results Search engine traffic     9.0
     Yahoo, search results   Search engine traffic     2.0
     Yandex Mobile           Search engine traffic    72.0
     Yandex, search results  Search engine traffic   185.0
     Yandex.Images           Search engine traffic     1.0
     Yandex: mobile app      Search engine traffic    35.0
7    Baidu, search results   Search engine traffic     6.0
     Google, search results  Search engine traffic  1572.0
     Google: mobile app      Search engine traffic     5.0
     Mail.ru, search results Search engine traffic     5.0
     Yandex Mobile           Search engine traffic    59.0
     Yandex, search results  Search engine traffic   175.0
     Yandex: mobile app      Search engine traffic    42.0
8    Baidu, search results   Search engine traffic     2.0
     Bing, search results    Search engine traffic     3.0
     DuckDuckGo              Search engine traffic     1.0
     Google, search results  Search engine traffic  1710.0
     Google: mobile app      Search engine traffic     5.0
     Mail.ru, search results Search engine traffic    12.0
     Yandex Mobile           Search engine traffic    61.0
     Yandex, search results  Search engine traffic   138.0
     Yandex.Images           Search engine traffic     1.0
     Yandex: mobile app      Search engine traffic    32.0
9    Baidu, search results   Search engine traffic     4.0
     Google, search results  Search engine traffic  1632.0
     Google: mobile app      Search engine traffic     7.0
     Mail.ru, search results Search engine traffic    10.0
     Yandex Mobile           Search engine traffic    75.0
     Yandex, search results  Search engi

In [91]:
df['utm_source'] = 'Органический трафик'
df

visits           utm_source
week utm_source              visit-id                                          
2    Google, search results  Search engine traffic  1994.0  Органический трафик
     Google: mobile app      Search engine traffic     8.0  Органический трафик
     Mail.ru, search results Search engine traffic     6.0  Органический трафик
     Yahoo, search results   Search engine traffic     1.0  Органический трафик
     Yandex Mobile           Search engine traffic    44.0  Органический трафик
     Yandex, search results  Search engine traffic    59.0  Органический трафик
     Yandex: mobile app      Search engine traffic    18.0  Органический трафик
3    Baidu, search results   Search engine traffic     1.0  Органический трафик
     Google, search results  Search engine traffic  2256.0  Органический трафик
     Google: mobile app      Search engine traffic     7.0  Органический трафик
     Mail.ru, search results Search engine traffic     7.0  Органический трафик
     Yandex Mobile           Search engine traffic    33.0  Органический трафик
     Yandex, search results  Search engine traffic    96.0  Органический трафик
     Yandex.Images           Search engine traffic     2.0  Органический трафик
     Yandex: mobile app      Search engine traffic    21.0  Органический трафик
4    DuckDuckGo              Search engine traffic     2.0  Органический трафик
     Google, search results  Search engine traffic  2112.0  Органический трафик
     Google: mobile app      Search engine traffic    11.0  Органический трафик
     Mail.ru, search results Search engine traffic    15.0  Органический трафик
     Ukr.net - Other         Search engine traffic     1.0  Органический трафик
     Yahoo, search results   Search engine traffic     1.0  Органический трафик
     Yandex Mobile           Search engine traffic    64.0  Органический трафик
     Yandex, search results  Search engine traffic   139.0  Органический трафик
     Yandex: mobile app      Search engine traffic    24.0  Органический трафик
5    Baidu, search results   Search engine traffic     2.0  Органический трафик
     Bing, search results    Search engine traffic     1.0  Органический трафик
     Ecosia                  Search engine traffic     1.0  Органический трафик
     Google, search results  Search engine traffic  1676.0  Органический трафик
     Google: mobile app      Search engine traffic     3.0  Органический трафик
     Mail.ru, search results Search engine traffic     6.0  Органический трафик
     Yahoo, search results   Search engine traffic     1.0  Органический трафик
     Yandex Mobile           Search engine traffic    68.0  Органический трафик
     Yandex, search results  Search engine traffic   166.0  Органический трафик
     Yandex.Images           Search engine traffic     2.0  Органический трафик
     Yandex: mobile app      Search engine traffic    19.0  Органический трафик
6    Baidu, search results   Search engine traffic     3.0  Органический трафик
     Google, search results  Search engine traffic  1458.0  Органический трафик
     Google: mobile app      Search engine traffic     5.0  Органический трафик
     Mail.ru, search results Search engine traffic     9.0  Органический трафик
     Yahoo, search results   Search engine traffic     2.0  Органический трафик
     Yandex Mobile           Search engine traffic    72.0  Органический трафик
     Yandex, search results  Search engine traffic   185.0  Органический трафик
     Yandex.Images           Search engine traffic     1.0  Органический трафик
     Yandex: mobile app      Search engine traffic    35.0  Органический трафик
7    Baidu, search results   Search engine traffic     6.0  Органический трафик
     Google, search results  Search engine traffic  1572.0  Органический трафик
     Google: mobile app      Search engine traffic     5.0  Органический трафик
     Mail.ru, search results Search engine traffic     5.0  Органический трафик
     Yandex Mobile           Search engine traffic  

In [92]:
df_utm = pd.concat([df_utm,df])

In [93]:
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr','vbr'] else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'SMS рекламные' if 
                                   x in ['tele2TommorowWON',
                                         'tele2Tommorow',
                                         'tele2FourDays',
                                         'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click'
                                          or x =='checkscan' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele' or x[:2]=='t2' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Органический трафик' if 
                                   x[:8]=='showcase' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'МТС' if 
                                   x[:2]=='mt' or x[:4]=='stre' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)
df_utm['utm_source'] = df_utm['utm_source'].apply(lambda x: 'Тестирование гипотез' if 
                                  x[:6]=='yandex' or x[:6]=='google' else x)
# # df['utm_source'] = dr.apply(lambda row: 'Органический трафик' if row['utm_source']=='Ссылочная масса' 
# #                         else row['utm_source'], axis=1)
# df['utm_source'] = dr.apply(lambda row: 'New (2Gis)' if row['utm_source']=='2gis' 
#                         else row['utm_source'], axis=1)
# df['utm_source'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['utm_source']=='Мобильное приложение' 
#                         else row['utm_source'], axis=1)

In [94]:
df_utm.columns = ['week','source_x','visits']

In [95]:
df_utm = df_utm.groupby(['week','source_x'],as_index=False).agg({'visits':'sum'})

In [96]:
#df = claims

In [97]:
#df =df[df['status_x']!='NEW']

In [98]:
since = date(2021, 1, 4)
till = date(2021, 12,31)

In [99]:
df = get_exact_claims(since, till, client)
df['requestId'] = df[df['partner']=='MoneyMan']['statusFields'].apply(mm_requestId)
dpbk = get_kibana_postback(es, since)
def float_apply(x):
    try:
        return float(x)
    except ValueError:
        return None
dpbk['payout'] = dpbk['payout'].apply(float_apply)
dpbk['payout'] = dpbk.apply(lambda x: x['payout']*1.2 if x['partnerName']=='admitad' else x['payout'], axis=1)
dpbk = dpbk[dpbk['status'].isin(['APPROVED', 'approved', 'ISSUED'])].sort_values('payout').drop_duplicates('transitionId', keep='last')
dr = pd.merge(left=df, right=dpbk, left_on='id', right_on='transitionId', how='left')
dc = get_kibana_claims(es, since,till)[['clickId', 'userId', 'utm_campaign', 'utm_medium', 'utm_content', 
                                        'utm_source', 'utm_term', 'gaId', 'yandexClientID', 'sourceGlobal']]
dr = pd.merge(left=dr, right=dc[dc['clickId'].notnull()].drop_duplicates('clickId'), on='clickId', how='left')

cre = cre_get(es, since)
dr['score'] = dr['userId_x'].map(cre).fillna(0).astype(int)

dr.to_pickle(f'reports/{datetime.strftime(since, "%B")}_claimReport.pickle.gz')
dc.to_pickle(f'reports/{datetime.strftime(since, "%B")}_claims.pickle.gz')

def AR_count(score, productType, status):
    koef = 1
    check = [2200, 1700, 1100, 5500]
    ARlist = [[0.00499500499500499, 0.00305343511450382, 0.0714285714285714, 0.0], 
              [0.00380818053596615, 0.00871508379888268, 0.0768345323741007, 0.0], 
              [0.00685378590078329, 0.0136418349605457, 0.0846340617412418, 0.0053475935828877], 
              [0.0111028867505552, 0.0255610972568579, 0.0980751604032997, 0.0116279069767442], 
              [0.00938967136150235, 0.0349985043374215, 0.126042632066728, 0.0], 
              [0.0109689213893967, 0.0407338308457712, 0.140204271123491, 0.0], 
              [0.0141884222474461, 0.0514050719671008, 0.113486842105263, 0.0188679245283019], 
              [0.0150753768844221, 0.0687361419068736, 0.0743243243243243, 0.0], 
              [0.00943396226415094, 0.0415430267062315, 0.0892857142857143, 0.0],
              [0.00943396226415094, 0.0415430267062315, 0.0892857142857143, 0.0]]
    
    if score is np.nan or status in ['NEW', 'ERROR']:
        return 0
    elif score > 10 and productType == 'CASH_LOAN':
        return round(ARlist[int(round(score, -1)/10)-1][0]*check[0]*koef, 2)
    elif score > 10 and productType == 'CREDIT_CARD':
        return round(ARlist[int(round(score, -1)/10)-1][1]*check[1]*koef, 2)
    elif score > 10 and productType == 'LOAN':
#         print(score, productType, status)
        return round(ARlist[int(round(score, -1)/10)-1][2]*check[2]*koef, 2)
    elif score > 10 and productType == 'REFINANCE':
        return round(ARlist[int(round(score, -1)/10)-1][3]*check[3]*koef, 2)
    else:
        return round(ARlist[int(round(score, -1)/10)-1][0]*check[0]*koef, 2)

dr['createdAt'] = pd.to_datetime(dr['createdAt'])
# dr['payout'] = dr['payout'].astype(float)
# dr['payout'] = dr.apply(lambda row: 3800 if row['partner']=='Альфа-Банк' 
#                         else row['payout'], axis=1)
# dr['payout'] = dr.apply(lambda row: 2550 if row['partner']=='Kredito24' 
#                         else row['payout'], axis=1)
# dr['productType'] = dr['product'].apply(lambda x: x['productType'])


dr['source'] = dr.apply(lambda row: 'Металинвест Cash' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)

dr = dr[dr['status_x']!='NEW']
dr['score'] = dr['score'].apply(lambda x: x+75 if x==0 else x)
dr['whence']=dr.apply(lambda row: row['whence_y'] if pd.notnull(row['whence_y']) else row['whence_x'],axis=1)
dr['source']=dr.apply(lambda row: row['utm_source_y'] if pd.notnull(row['utm_source_y']) else row['whence'],axis=1)
dr['medium']=dr.apply(lambda row: row['utm_medium_y'] if pd.notnull(row['utm_medium_y']) else row['whence'],axis=1)

sms = dr.groupby('source').agg({'_id':'count'}).to_excel('sms.xlsx')

dr['source']=dr.apply(lambda row: row['utm_source_y'] if 
                      pd.notnull(row['utm_source_y']) else row['whence'],axis=1)
dr['source']=dr.apply(lambda row: row['sourceGlobal'] if 
                      pd.notnull(row['sourceGlobal']) else row['source'],axis=1)
dr['source']=dr.apply(lambda row: 'Город Тройка' if 
                      row['utm_content']=='gorodtroika' else row['source'],axis=1)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr'] else x)
dr['source'] = dr['source'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x in ['tele2TommorowWON',
                                         'tele2Tommorow',
                                         'tele2FourDays',
                                         'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
dr['source'] = dr['source'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
dr['source'] = dr['source'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
dr['source'] = dr['source'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click'  else x)
dr['source'] = dr['source'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
dr['source'] = dr['source'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele' else x)
dr['source'] = dr['source'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
dr['source'] = dr['source'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
dr['source'] = dr['source'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
dr['source'] = dr['source'].apply(lambda x: 'Ссылочная масса' if 
                                   x[:8]=='showcase' else x)
dr['source'] = dr['source'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content-text' else x)
dr['source'] = dr['source'].apply(lambda x: 'МТС \\ баннер + СМС' if 
                                   x[:2]=='mt' or x[:6]=='stream' else x)
dr['source'] = dr['source'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)
dr['source'] = dr['source'].apply(lambda x: 'Тестирование гипотез' if 
                                  x[:6]=='yandex' or x[:6]=='google' else x)
dr['source']=dr.apply(lambda row: 'Тестирование гипотез' if 
                      row['utm_source_y']=='yandex' or row['utm_source_y']=='google' else row['source'],axis=1)

dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)
dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5ef4473acf22a90001010445' 
                        else row['source'], axis=1)


def get_kibana_reg(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 1000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/1000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-1000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID','currentPage',
#                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
#                   'transaction_id', 'clickId', 
                'sourceGlobal']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

regs = get_kibana_reg(es, since,till)

# dr['utm_source_x'].value_counts()

regs['currentPage1'] = regs['currentPage'].apply(lambda x: x[:5] if pd.notnull(x) else np.nan)
regs['currentPage2'] = regs['currentPage'].apply(lambda x: x[:9] if pd.notnull(x) else np.nan)

dr=pd.merge(left=dr, right=regs, how='left', on='userId')

# odin = dr[dr['dt_reg'].notnull()]
# odin=odin[odin['product_type']=='LOAN']
# odin['userId'].nunique()

dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Android_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Huawei_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Apple_mobile' 
                        else row['source_x'], axis=1)

dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage1']=='/news' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage2']=='/articles' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Органический трафик' if row['source_x']=='Ссылочная масса' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'New (2Gis)' if row['source_x']=='2gis' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['source_x']=='Мобильное приложение' 
                        else row['source_x'], axis=1)
# Костыль

dr['status_x'] = dr.apply(lambda x: 'APPROVED' if
                        pd.notnull(x['status_y']) and x['status_y'] in(['APPROVED', 'approved', 'ISSUED']) 
                          else x['status_x'], axis=1)
                          


38036


INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:1.205s]


572026; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.295s]


562026; 

...

2454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.041s]


1454; 

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.150s]


454; 

In [100]:
dr['dt']  =pd.to_datetime(dr['dt'])
dr['week'] = (dr['dt'].dt.week)+1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [101]:
#df['week']= (pd.to_datetime(df['dt']).dt.week)+1
dr_fin  =dr[~dr['source_x'].isin(['SMS рекламные','Email','Vbr.ru ссылки',
                               'Мобильное приложение 1'])].groupby(['week'],
                                                                   as_index=False).agg({'userId_x':'count'})


In [102]:
# df_utm1=df_utm.groupby(['week'],as_index=False).agg({'visits':'sum'})

In [103]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           # 'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][4]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            #'gender':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df = df.groupby(['week'],as_index=False).agg({'visits':'sum'})

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [104]:
traf_claim= df.merge(dr_fin,how='left',on=['week'])

In [105]:
#traf_claim = traf_claim[~traf_claim['source_x'].isin(['Email','Vbr.ru ссылки','SMS рекламные','Bankiros.ru ссылки'])]

In [106]:
# traf_claim = traf_claim[traf_claim['visits']>30]
# traf_claim = traf_claim.fillna(0)


In [107]:
traf_claim['cr']= ((traf_claim['userId_x']/traf_claim['visits'])*100).round(2)
traf_claim.to_excel('Еженедельная презентация/траф_в_заявку.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe
# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(23) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/траф_в_заявку.xlsx')
set_with_dataframe(worksheet, df1) 

In [108]:
def get_kibana_click_broker(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(date.today()+timedelta(days=1), datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"CLAIM_BROKER"}},
#                          {"match_phrase":{"params.whence.keyword":{"query": whence }}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}
             # ,"_source": ["_id","params.clickId", "params.pid", "params.reqid", "params.wid","timestamp"]
              }
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['_id','pid','clickId','wid','reqId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

dobs= get_kibana_click_broker(es,since,till)

dobs = dobs[['clickId','pid','reqId','date']]
dobs= dobs[~dobs['clickId'].isin(['gtm-msr.appspot.com','test'])]
dobs['week']=(pd.to_datetime(dobs['date']).dt.week)+1
dobs

INFO:elasticsearch:POST http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:0.039s]


968; 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


,clickId,pid,reqId,date,week
96,9m7Z1ViYPnZebMj,cityads,7125135,2021-11-28,48
97,9m7Z1ViYQvZiAv3,cityads,7125138,2021-11-28,48
98,9m7Z1ViYPnZebMj,cityads,7125142,2021-11-28,48
104,9m7Z1VjDC8Z5iFh,cityads,7132146,2021-11-30,49
105,9m7Z1VjF0sZiC1G,cityads,7132495,2021-11-30,49
106,9m7Z1VjFN3ZEDEw,cityads,7132673,2021-11-30,49
107,9m7Z1VjKlvZiBAG,cityads,7133644,2021-11-30,49
108,9m7Z1Vjdw0ZA84N,cityads,7127095,2021-11-29,49
109,9m7Z1VjhNmZ9IPD,cityads,7128421,2021-11-29,49
110,9m7Z1VjlIaZrlj1,cityads,7129452,2021-11-29,49


In [109]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:UTMSource,ym:s:startURLPathLevel2",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][4]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            'start':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week1'] =(df['date'].dt.week)+1
df_utm = df.groupby(['week1','utm_source','start'],as_index=False).agg({'visits':'sum'})
df_utm

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


,week1,utm_source,start,visits
0,2,admitad,https://landings.odobrim.ru/bcs_broker,549.0
1,2,admitad,https://landings.odobrim.ru/bcs_broker#,1.0
2,2,admitad,https://landings.odobrim.ru/mikrozajmy?,32.0
3,2,admitad,https://odobrim.ru/podbor-kreditov-vidacha?,299.0
4,2,advcake,https://landings.odobrim.ru/bcs_broker/,56.0
...,...,...,...,...
24735,52,tele2,https://landings.odobrim.ru/tele2,17.0
24736,52,tele2,https://landings.odobrim.ru/tele2#,1.0
24737,52,yandex,https://landings.odobrim.ru/tele2,2.0
24738,52,yandex,https://odobrim.ru/deposit,54.0


In [110]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:visits",
    dimensions = "ym:s:date,ym:s:searchEngine,ym:s:startURLPathLevel2",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000
# filters = "ym:s:UTMSource=='smsTopBP'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'utm_source':result[i]["dimensions"][1]["name"],
#             'visit-id':result[i]["dimensions"][3]["name"],
            #'area':result[i]["dimensions"][4]["name"],
#             'users':result[i]["metrics"][0],
             'visits':result[i]["metrics"][0],
            'start':result[i]["dimensions"][2]["name"],
#'avgVisitDurationSeconds':result[i]["dimensions"][3]["name"],
           #'yandexClientId':result[i]["dimensions"][5]["name"]
            #'startURL':result[i]["dimensions"][7]
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week1'] =(df['date'].dt.week)+1
df['utm_source'] = 'seo'
df_seo = df.groupby(['week1','utm_source','start'],as_index=False).agg({'visits':'sum'})
df_seo1 =df_seo.groupby(['week1','utm_source','start'],as_index=False).agg({'visits':'sum'}) 
df_seo1

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


,week1,utm_source,start,visits
0,2,seo,https://landings.odobrim.ru/bcs_broker,570.0
1,2,seo,https://landings.odobrim.ru/bcs_broker#,1.0
2,2,seo,https://landings.odobrim.ru/bcs_broker/,57.0
3,2,seo,https://landings.odobrim.ru/bcs_broker?,2.0
4,2,seo,https://landings.odobrim.ru/broker,48.0
...,...,...,...,...
26546,52,seo,https://odobrim.ru/results,1.0
26547,52,seo,https://odobrim.ru/scs,1.0
26548,52,seo,https://odobrim.ru/unsubscribe?,1.0
26549,52,seo,https://odobrim.ru/vhod,1.0


In [111]:
df_utm

,week1,utm_source,start,visits
0,2,admitad,https://landings.odobrim.ru/bcs_broker,549.0
1,2,admitad,https://landings.odobrim.ru/bcs_broker#,1.0
2,2,admitad,https://landings.odobrim.ru/mikrozajmy?,32.0
3,2,admitad,https://odobrim.ru/podbor-kreditov-vidacha?,299.0
4,2,advcake,https://landings.odobrim.ru/bcs_broker/,56.0
...,...,...,...,...
24735,52,tele2,https://landings.odobrim.ru/tele2,17.0
24736,52,tele2,https://landings.odobrim.ru/tele2#,1.0
24737,52,yandex,https://landings.odobrim.ru/tele2,2.0
24738,52,yandex,https://odobrim.ru/deposit,54.0


In [112]:
df_merge= pd.concat([df_utm,df_seo1])

In [113]:
df_merge['utm_source'].unique()

array(['admitad', 'advcake', 'advertise', 'ais_google', 'ais_yandex',
       'bank-zenit', 'content', 'content-text', 'cpahub', 'cpahub2',
       'email_promo', 'email_trigger', 'exbico', 'google', 'inst',
       'kupikupon', 'landing_odobrim', 'leadgid', 'leadgid3', 'leads2',
       'megafon_channels', 'mt', 'push-notification', 'pushworld',
       'rubid2', 'sms_promo', 'sms_trigger', 'tele2', 'tele_2', 'vbr',
       'yandex', 'yandex.zen', '{utm_source}', 'callcenter', 'email',
       'fb', 'mytarget', 'yandex.', 'lifehacker', 'sberometer',
       'bankiros', 'kupibonus', 'mail.ru', 'criteo', 'ok.ru', 'skype',
       'telegram', 'text', 'vk.com', 'whatsapp', 'leadgid2', 'leads',
       'tiktok', 'yandex.ru', 'bcs_sms', 'esign', 'smsSignal', 'smsBP',
       'smsFebr', 'smsK0324', 'smsP0324', 'odobrim', 'sms_bcs', 'hey',
       'smsTopBP', 'checkscan', 'i', 'evadav', 'landing_pdk', 'propeller',
       'admitad,google', 'test', 'lostBasket', 'get4click', 'gmail.com',
       'mnogoru', 

In [114]:
df_merge

,week1,utm_source,start,visits
0,2,admitad,https://landings.odobrim.ru/bcs_broker,549.0
1,2,admitad,https://landings.odobrim.ru/bcs_broker#,1.0
2,2,admitad,https://landings.odobrim.ru/mikrozajmy?,32.0
3,2,admitad,https://odobrim.ru/podbor-kreditov-vidacha?,299.0
4,2,advcake,https://landings.odobrim.ru/bcs_broker/,56.0
...,...,...,...,...
26546,52,seo,https://odobrim.ru/results,1.0
26547,52,seo,https://odobrim.ru/scs,1.0
26548,52,seo,https://odobrim.ru/unsubscribe?,1.0
26549,52,seo,https://odobrim.ru/vhod,1.0


In [115]:
df_f= df_merge[df_merge['visits']>50]
df_f.to_excel('Еженедельная презентация/startPage.xlsx')
# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(17) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/startPage.xlsx')
set_with_dataframe(worksheet, df1) 

In [116]:
df_f

,week1,utm_source,start,visits
0,2,admitad,https://landings.odobrim.ru/bcs_broker,549.0
3,2,admitad,https://odobrim.ru/podbor-kreditov-vidacha?,299.0
4,2,advcake,https://landings.odobrim.ru/bcs_broker/,56.0
506,2,advcake,https://odobrim.ru/podbor-kreditov-vidacha?,151.0
510,2,ais_google,https://odobrim.ru/car-loan,185.0
...,...,...,...,...
26469,51,seo,https://odobrim.ru/products/,97.0
26470,51,seo,https://odobrim.ru/refinance,151.0
26477,51,seo,https://odobrim.ru/scs,255.0
26525,52,seo,https://odobrim.ru/deposit,55.0


In [117]:
df_f1 = df_f
df_f1.columns = ['week2','utm_source1','start','visits']
df_f1.to_excel('Еженедельная презентация/startPage1.xlsx')
# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1W8_XtZaL8xqX1y_pVLN5dCM-zToXKzU3sMuYUT7JYak')
worksheet = sh.get_worksheet(18) #-> 0 - лист один, 1 - второй и тд.
# Выбор датафрейма
df1 = pd.read_excel('Еженедельная презентация/startPage1.xlsx')
set_with_dataframe(worksheet, df1)

In [118]:
df_f1

,week2,utm_source1,start,visits
0,2,admitad,https://landings.odobrim.ru/bcs_broker,549.0
3,2,admitad,https://odobrim.ru/podbor-kreditov-vidacha?,299.0
4,2,advcake,https://landings.odobrim.ru/bcs_broker/,56.0
506,2,advcake,https://odobrim.ru/podbor-kreditov-vidacha?,151.0
510,2,ais_google,https://odobrim.ru/car-loan,185.0
...,...,...,...,...
26469,51,seo,https://odobrim.ru/products/,97.0
26470,51,seo,https://odobrim.ru/refinance,151.0
26477,51,seo,https://odobrim.ru/scs,255.0
26525,52,seo,https://odobrim.ru/deposit,55.0


In [119]:
def get_kibana_click_broker(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(date.today()+timedelta(days=1), datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"CLICK_BROKER"}},
#                          {"match_phrase":{"params.whence.keyword":{"query": whence }}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}
             # ,"_source": ["_id","params.clickId", "params.pid", "params.reqid", "params.wid","timestamp"]
              }
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['_id','pid','clickId','wid',]:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    
    return dr